In [22]:
from ciphers.gpu import present80_gpu
from ciphers.cpu import present80 as present80_cpu

from ciphers.cpu import speck3264 as speck3264_cpu
from ciphers.cpu import speck64128 as speck64128_cpu

from ciphers.gpu import speck3264_gpu
from ciphers.gpu import speck64128_gpu

## Test for pure encryption

### Present

In [ ]:
# present80_cpu.benchmark_present80_numpy(batch_size=2000000)

In [ ]:
# present80_gpu.benchmark_gpu(batch_size=2000000)

### Speck3264

In [2]:
# speck inputdifference:
0x8080

32896

In [2]:
speck3264_cpu.check_testvectors()

In [3]:
speck3264_gpu.check_testvectors()

Final ciphertext (bit-level path): ['0xa868', '0x42f2']
Match expected ciphertext [0xa868, 0x42f2]?
True


In [4]:
speck3264_cpu.benchmark_numpy(batch_size=10_000_000)

10000000 blocks, 22 rounds: 15.8075s, tốc độ = 632,610.15 blocks/sec


In [5]:
speck3264_gpu.benchmark_gpu(batch_size=10_000_000)

10000000 blocks in 5.8010s, speed = 1723831.68 blocks/sec.


### Speck64128

In [23]:
speck64128_cpu.benchmark_numpy(batch_size=1_000_000)

1000000 blocks, 22 rounds: 3.9952s, tốc độ = 250,300.83 blocks/sec


In [24]:
speck64128_gpu.benchmark_gpu(batch_size=1_000_000)

1000000 blocks in 0.2099s, speed = 4765231.10 blocks/sec.


## Test for data generator (encryption x2 -> XORing)

### simple make_train_data (C1,C2)

#### CPU - PRESENT

In [ ]:
from generator import cipherpair_generator_cpu as cc

In [ ]:
cipher_name = "present" 
encryption_function = present80_cpu.encrypt
plain_bits = present80_cpu.plain_bits
key_bits = present80_cpu.key_bits
num_samples = 1_000_000
nr = 32

delta = cc.integer_to_binary_array(0x90000000000, plain_bits)
delta_key = 0
delta_plain = delta[:plain_bits]

data_generator = lambda num_samples, nr: cc.make_train_data(
    encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
)

# maybe can put this func into \test ?
C_gpu, Y_gpu = cc.benchmark_make_train_data_cpu(encryption_function, num_samples, nr, plain_bits, key_bits)


[CPU/NumPy] Tạo 1000000 mẫu: 99.1131s (10,089.48 mẫu/giây)


In [ ]:
# maybe can put these func into \utils ?
cc.save_ciphertexpair_dataset_npz(C_gpu,Y_gpu,cipher_name=cipher_name,num_rounds=nr,num_samples=num_samples)

✅ Saved present dataset to ../../data\present_ciphertextpair_present_round32_samples1000000.npz


### GPU PRESENT

In [2]:
from generator import cipherpair_generator_gpu as cg

In [ ]:
cipher_name = "present" 
encryption_function = present80_gpu.encrypt
plain_bits = present80_gpu.plain_bits
key_bits = present80_gpu.key_bits
num_samples = 1_000_000
nr = 32

delta = cg.integer_to_binary_array(0x90000000000, plain_bits)
delta_key = 0
delta_plain = delta[:plain_bits]

data_generator = lambda num_samples, nr: cg.make_train_data(
    encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
)

# maybe can put this func into \test ?
C_gpu, Y_gpu = cp.benchmark_make_train_data_gpu(encryption_function, num_samples, nr, plain_bits,key_bits)


[GPU/CuPy] Tạo 1000000 mẫu: 6.6792s (149,718.28 mẫu/giây)


In [ ]:
# maybe can put these func into \utils ?
cc.save_ciphertexpair_dataset_npz(C_gpu,Y_gpu,cipher_name=cipher_name+"_gpu",num_rounds=nr,num_samples=num_samples)

✅ Saved present_gpu dataset to ../../data\present_gpu_ciphertextpair_present_gpu_round32_samples1000000.npz


### CPU SPECK

In [8]:
from generator import cipherpair_generator_cpu as cc
from generator import cipherpair_generator_gpu as cg

In [20]:
from ciphers.gpu import speck3264_gpu
from ciphers.cpu import speck3264 as speck3264_cpu

from ciphers.cpu import speck64128 as speck64128_cpu
from ciphers.gpu import speck64128_gpu

In [10]:
cipher_name = "speck" 
encryption_function = speck3264_cpu.encrypt
plain_bits = speck3264_cpu.plain_bits
key_bits = speck3264_cpu.key_bits
num_samples = 10_000_000
nr = speck3264_cpu.rounds

delta = cc.integer_to_binary_array(0x80000, plain_bits)
delta_key = 0
delta_plain = delta[:plain_bits]

data_generator = lambda num_samples, nr: cc.make_train_data(
    encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
)

# maybe can put this func into \test ?
C_gpu, Y_gpu = cc.benchmark_make_train_data_cpu(encryption_function, num_samples, nr, plain_bits, key_bits)


[CPU/NumPy] Tạo 10000000 mẫu: 25.0855s (398,636.99 mẫu/giây)


In [11]:
C_gpu.shape

(10000000, 64)

### GPU SPECK

In [13]:
cipher_name = "speck" 
encryption_function = speck3264_gpu.encrypt
plain_bits = speck3264_gpu.plain_bits
key_bits = speck3264_gpu.key_bits
num_samples = 10_000_000
nr = speck3264_gpu.rounds

delta = cg.integer_to_binary_array(0x40000, plain_bits)
delta_key = 0
delta_plain = delta[:plain_bits]

data_generator = lambda num_samples, nr: cg.make_train_data(
    encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
)

# maybe can put this func into \test ?
C_gpu, Y_gpu = cg.benchmark_make_train_data_gpu(encryption_function, num_samples, nr, plain_bits, key_bits)


[GPU/CuPy] Tạo 10000000 mẫu: 45.2043s (221,217.70 mẫu/giây)


In [14]:
cipher_name = "speck" 
encryption_function = speck64128_gpu.encrypt
plain_bits = speck64128_gpu.plain_bits
key_bits = speck64128_gpu.key_bits
num_samples = 1_000_000
nr = speck64128_gpu.rounds

delta = cg.integer_to_binary_array(0x40000, plain_bits)
delta_key = 0
delta_plain = delta[:plain_bits]

data_generator = lambda num_samples, nr: cg.make_train_data(
    encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
)

# maybe can put this func into \test ?
C_gpu, Y_gpu = cg.benchmark_make_train_data_gpu(encryption_function, num_samples, nr, plain_bits, key_bits)

[GPU/CuPy] Tạo 1000000 mẫu: 1.3625s (733,919.55 mẫu/giây)


In [25]:
cipher_name = "speck" 
encryption_function = speck64128_cpu.encrypt
plain_bits = speck64128_cpu.plain_bits
key_bits = speck64128_cpu.key_bits
num_samples = 1_000_000
nr = speck64128_cpu.rounds

delta = cc.integer_to_binary_array(0x40000, plain_bits)
delta_key = 0
delta_plain = delta[:plain_bits]

data_generator = lambda num_samples, nr: cc.make_train_data(
    encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
)

# maybe can put this func into \test ?
C_gpu, Y_gpu = cc.make_train_data(encryption_function, num_samples, nr, plain_bits, key_bits)

IndexError: index -1 is out of bounds for axis 0 with size 0

## Test training (one round)

### DbitNet

In [ ]:
import tensorflow as tf
from models import dbitnet

PRESENT GPU

In [ ]:
from models import train_nets as tn

In [ ]:
input_size = plain_bits
model = dbitnet.make_model(2 * input_size)
optimizer = tf.keras.optimizers.Adam(amsgrad=True)

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
